# News Web-Crawling

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time

# 사용자 에이전트 설정
user_agent_string = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

# 웹 페이지 요청 및 파싱 함수
def make_request(url):
    response = requests.get(url, headers={'User-Agent': user_agent_string})
    if response.status_code == 200:
        content_type = response.headers['Content-Type'].lower()
        if "euc-kr" in content_type or ".html" in url:
            response.encoding = 'euc-kr'
        return BeautifulSoup(response.content, 'html.parser')
    else:
        return None

In [23]:
import pandas as pd

url_list_all = pd.read_csv('MediaTopic_R/article.url.csv')
url_list_all.head()

,ID,news,URL
0,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...
1,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...
2,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...
3,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...
4,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...


In [24]:
url_list_all[url_list_all['URL'].str.contains('news.join')]

,ID,news,URL
334,1.100901e+06,중앙일보,https://news.joins.com/article/olink/3176644
335,1.100901e+06,중앙일보,https://news.joins.com/article/olink/3185500
336,1.100901e+06,중앙일보,https://news.joins.com/article/olink/3276221
337,1.100901e+06,중앙일보,https://news.joins.com/article/olink/3279497


In [25]:
url_list_all = url_list_all[url_list_all['URL'].str.contains('news.join') == False]

In [6]:
print(url_list_all['ID'].nunique())
print(url_list_all['news'].unique())

625
['경향신문' '동아일보' '중앙일보' '한겨레']


In [5]:
url_list_all['URL'].nunique()

637

## 경향신문


In [ ]:
url_list_kh = url_list_all[url_list_all['news']=='경향신문']
url_list_kh['content'] = ""
url_list_kh.head()

In [ ]:
## 
content_list = []
i = 0
for url in url_list_kh['URL']:
    webpage = make_request(url)
    if webpage:
        if webpage.select('div.article_cont'):
            article_text = ' '.join([p.get_text() for p in webpage.select('div.article_cont')])
        else:
            article_text = ' '.join([p.get_text() for p in webpage.select('div.art_body')])

        article_text = re.sub(r'\n|\r|\t', '', article_text).strip()
        article_text = re.sub(r'createIframe\(.*?\);\s*', '', article_text)
   
    url_list_kh['content'][i] = article_text
    time.sleep(1)
    print(url, url_list_kh['content'][i])
    i += 1

In [ ]:
url_list_kh.to_csv('url_list_kh.csv', encoding='utf-8-sig')
url_list_kh.head()

## 동아일보


In [ ]:
url_list_da = url_list_all[url_list_all['news']=='동아일보'].reset_index(drop=True)
url_list_da['content'] = ""
url_list_da.head()

In [ ]:
content_list = []
for url in url_list_da['URL']:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.select_one('.article_txt')
    article_text = re.sub(r'\n|\r|\t', '', content.text).strip()
    article_text = re.sub(r'createIframe\(.*?\);\s*', '', article_text)
    content_list.append(article_text)
    time.sleep(1)
    
url_list_da['content'] = content_list
url_list_da  

In [ ]:
url_list_da.to_csv('url_list_da.csv', encoding='utf-8-sig')
url_list_da.head()

## 중앙일보


In [ ]:
url_list_ja = url_list_all[url_list_all['news']=='중앙일보'].reset_index(drop=True)
url_list_ja['content'] = ""
url_list_ja.head()

In [ ]:
content_list = []
i = 0
for url in url_list_ja['URL']:
    webpage = make_request(url)
    if webpage:
        article_elements = webpage.select('div.article_body')
        article_text = ' '.join([p.get_text() for p in article_elements[0].select('p')])
        content_list.append(article_text)
        time.sleep(1)
        # paragraphs = webpage.find_all('p')
        # texts = [p.get_text() for p in paragraphs]
        # article_text = max(texts, key=len, default="")
        # content_list.append(article_text)
    else:
        article_text = ""
        content_list.append(article_text)
        time.sleep(1)
url_list_ja['content'] = content_list
url_list_ja      


In [ ]:
url_list_ja.to_csv('url_list_ja.csv', encoding='utf-8-sig')
url_list_ja.head()

## 한겨레


In [ ]:
url_list_han = url_list_all[url_list_all['news']=='한겨레'].reset_index(drop=True)
url_list_han['content'] = ""
url_list_han.head()

In [ ]:
# url_list_han의 'url' 컬럼의 값들을 순회하면서 기사 본문을 추출
i = 0
for url in url_list_han['URL']:
    webpage = make_request(url)
    if webpage:
        if webpage.select_one('div.text') == None:
            article_elements = webpage.select('div.news_text01')
            article_text = ' '.join([element.get_text() for element in article_elements])
            article_text = re.sub(r'\n|\r|\t', '', article_text).strip()
            article_text = re.sub(r'function getPoll().*$', '', article_text)
        else:
            article_text = webpage.select_one('div.text').get_text()
            article_text = re.sub(r'\n|\r|\t', '', article_text).strip()
            article_text = re.sub(r'function getPoll().*$', '', article_text)
    else:
        article_text = ""
        time.sleep(1)
        
    url_list_han['content'][i] = article_text
    time.sleep(1)
    print(url, url_list_han['content'][i])
    i += 1

In [ ]:
url_list_han.to_csv('url_list_han.csv', encoding='utf-8-sig')
url_list_han.head()

## 네이버뉴스

In [ ]:
url_list_naver = url_list_all[url_list_all['URL'].str.contains('naver')].reset_index(drop=True)
url_list_naver['content'] = ""
url_list_naver.head()

In [ ]:
url = url_list_naver['URL'][0]
i = 0
for url in url_list_naver['URL']:
    webpage = make_request(url)
    if webpage:
        article_elements = webpage.select('div.newsct_article')
        article_text = ' '.join([element.get_text() for element in article_elements])
        article_text = re.sub(r'\n|\r|\t', '', article_text).strip()
        article_text = re.sub(r'function getPoll().*$', '', article_text)
        
    url_list_naver['content'][i] = article_text
    time.sleep(1)
    print(url, url_list_naver['content'][i])
    i += 1

In [ ]:
url_list_naver.to_csv('url_list_naver.csv', encoding='utf-8-sig')
url_list_naver.head()

# Merge

In [42]:
url_list_da = pd.read_csv('url_list_da.csv')
url_list_kh = pd.read_csv('url_list_kh.csv')

In [ ]:
url_list_all = pd.concat([url_list_han, url_list_ja, url_list_da, url_list_kh, url_list_naver])
url_list_all = url_list_all.reset_index(drop=True)
url_list_all

In [44]:
url_list_all.to_csv('url_list_all_fixed.csv', encoding='utf-8-sig')